In [19]:
import numpy as numpy
import pandas as pd
import seaborn as sns #for plotting
import matplotlib.pyplot as plt
import random
import os
from scipy.stats import zscore
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import requests
from Bio.PDB import PDBParser, NeighborSearch
from propy.PseudoAAC import GetPseudoAAC, _Hydrophobicity, _hydrophilicity, _residuemass
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import glob
from propy.PseudoAAC import GetPseudoAAC, _Hydrophobicity, _hydrophilicity, _residuemass
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from scipy.stats import chi2_contingency

In [20]:
# Wasser aus ctla4_interface_residues entfernen
# Datei laden
df = pd.read_csv("data/ctla4/ctla4_interface_residues.csv")

# Zeilen mit 'HOH' in einer beliebigen Spalte entfernen (z. B. in Spalte 3)
df_clean = df[~df.iloc[:, 2].astype(str).str.contains("HOH")]

# Gesäuberte Datei speichern
df_clean.to_csv("data/ctla4/ctla4_interface_residues_clean.csv", index=False)

**Chi2 Test**
Prüft, ob sich die Zeilenverteilungen signifikant unterscheiden.

In [22]:
var = 'paratope'
files = {
    'CTLA4': f'data/ctla4/ctla4_{var}_residues.csv',
    'KIR': f'data/kir/kir_{var}_residues.csv',
    'LAG3': f'data/lag3/lag3_{var}_residues.csv',
    'PD1': f'data/pd1/pd1_{var}_residues.csv',
    'PDL1': f'data/pdl1/pdl1_{var}_residues.csv',
    'TIM3': f'data/tim3/tim3_{var}_residues.csv',
}

# Häufigkeitstabellen erstellen
dfs = []
for checkpoint, path in files.items():
    df = pd.read_csv(path)
    counts = df['residue_name'].value_counts()
    counts.name = checkpoint
    dfs.append(counts)

# Kontingenztabelle erzeugen (Residuen = Spalten, Checkpoints = Zeilen)
contingency_table = pd.DataFrame(dfs).fillna(0).astype(int)

print("Kontingenztabelle:\n", contingency_table)

# Chi²-Test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("\nChi²-Wert:", chi2)
print("p-Wert:", p)
print("Freiheitsgrade:", dof)

# Erwartete Häufigkeiten und Residuen
expected_df = pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns)
residuals = (contingency_table - expected_df) / (expected_df**0.5)

print("\nStandardisierte Residuen:\n", residuals)


Kontingenztabelle:
 residue_name  TYR  SER  GLY  TRP  ASN  THR  HIS  LEU  VAL  PHE  ...  ARG  GLU  \
CTLA4          60   42   36   22   19   11   10   10    8    8  ...    4    4   
KIR             4    6    1    1    1    2    0    0    1    2  ...    0    1   
LAG3            6    2   10    0    2    8    0    2    0    1  ...    0    0   
PD1            99   64   54   16   41   30    9   19    8   22  ...   19    2   
PDL1           30   34   17   15    5   12    4    8    4    8  ...   10    6   
TIM3            4    6    3    2    1    1    0    0    3    1  ...    0    0   

residue_name  GLN  PRO  FLC  ALA  ILE  MET  EDO  SO4  
CTLA4           4    3    1    1    1    0    0    0  
KIR             0    1    0    0    1    0    0    0  
LAG3            2    2    0    2    2    0    0    0  
PD1             3    5    0   11   12    5    1    0  
PDL1            0    8    0    6    4    0    0    1  
TIM3            0    0    0    4    0    0    0    0  

[6 rows x 22 columns]

Chi

In [23]:
files = {
    'CTLA4': 'data/ctla4/ctla4_epitope_residues.csv',
    'KIR': 'data/kir/kir_epitope_residues.csv',
    'LAG3': 'data/lag3/lag3_epitope_residues.csv',
    'PD1': 'data/pd1/pd1_epitope_residues.csv',
    'PDL1': 'data/pdl1/pdl1_epitope_residues.csv',
    'TIM3': 'data/tim3/tim3_epitope_residues.csv',
}

# Häufigkeitstabellen erstellen
dfs = []
for checkpoint, path in files.items():
    df = pd.read_csv(path)
    counts = df['residue_name'].value_counts()
    counts.name = checkpoint
    dfs.append(counts)

# Kontingenztabelle erzeugen (Residuen = Spalten, Checkpoints = Zeilen)
contingency_table = pd.DataFrame(dfs).fillna(0).astype(int)

print("Kontingenztabelle:\n", contingency_table)

# Chi²-Test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("\nChi²-Wert:", chi2)
print("p-Wert:", p)
print("Freiheitsgrade:", dof)

# Erwartete Häufigkeiten und Residuen
expected_df = pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns)
residuals = (contingency_table - expected_df) / (expected_df**0.5)

print("\nStandardisierte Residuen:\n", residuals)


Kontingenztabelle:
 residue_name  TYR  PRO  GLU  LEU  MET  THR  ILE  LYS  VAL  GLN  ...  GLY  SER  \
CTLA4          35   32   31   29   22   18   18   13   11   11  ...   10    8   
KIR             2    1    0    4    0    1    2    2    0    0  ...    1    2   
LAG3            1    4    4    1    0    0    0    0    2    5  ...    3    7   
PD1             8   37   26   21    2   13   30   31   15   27  ...    5   43   
PDL1           22    0   16    5   15    3   11    8   14    7  ...   11    6   
TIM3            0    3    2    0    1    0    3    2    2    1  ...    3    0   

residue_name  ALA  HIS  ASN  ASP  PHE  CYS  TRP  NAG  
CTLA4           7    6    3    2    2    1    0    0  
KIR             0    3    0    2    2    0    0    0  
LAG3            3    0    0    0    2    0    0    0  
PD1            36    0   17   21   15    0    3    1  
PDL1           21    7    6   22    3    0    1    0  
TIM3            1    0    2    1    1    2    0    0  

[6 rows x 21 columns]

Chi

**Fischer Exact Test**
Prüft, ob **zwei kategoriale Variablen** in einer **2x2-Kreuztabelle** **unabhängig voneinander sind** – oder ob es eine **statistisch signifikante Verbindung** zwischen ihnen gibt. **Kommt Aminosäure X signifikant häufiger im Paratope vor als im Epitope? Oder ist der Unterschied rein zufällig?**

In [24]:
import pandas as pd
from scipy.stats import fisher_exact

# Beispielhaft für CTLA4-Datensatz – passe die Pfade bei Bedarf an
epitope_path = "data/ctla4/ctla4_epitope_residues.csv"
paratope_path = "data/ctla4/ctla4_paratope_residues.csv"

# CSV-Dateien laden
df_epi = pd.read_csv(epitope_path)
df_para = pd.read_csv(paratope_path)

# Nur Spalten mit Aminosäure-Infos auswählen
# Annahme: Spalte heißt z. B. "residue_name" oder ähnlich – ggf. anpassen!
residue_col = "residue_name"

# Aminosäurelisten
aa_list = sorted(set(df_epi[residue_col]) | set(df_para[residue_col]))

# Ergebnisse speichern
results = []

for aa in aa_list:
    # Zähle für diese Aminosäure:
    epi_aa = (df_epi[residue_col] == aa).sum()
    epi_not_aa = (df_epi[residue_col] != aa).sum()
    para_aa = (df_para[residue_col] == aa).sum()
    para_not_aa = (df_para[residue_col] != aa).sum()

    # 2x2-Tabelle
    table = [[para_aa, para_not_aa],
             [epi_aa, epi_not_aa]]

    # Fisher-Test
    _, p = fisher_exact(table)

    results.append({
        "amino_acid": aa,
        "paratope_count": para_aa,
        "epitope_count": epi_aa,
        "p_value": p
    })

# Als DataFrame ausgeben
results_df = pd.DataFrame(results)

# Nach Signifikanz sortieren
results_df = results_df.sort_values("p_value")

# Ausgabe
print(results_df)

# Optional: Als CSV speichern
results_df.to_csv("fisher_test_ctla4.csv", index=False)


   amino_acid  paratope_count  epitope_count       p_value
16        SER              42              8  7.347379e-08
18        TRP              22              0  8.086415e-08
15        PRO               3             32  2.446741e-07
13        MET               0             22  3.563456e-07
7         GLU               4             31  2.186035e-06
8         GLY              36             10  2.199136e-05
10        ILE               1             18  6.438259e-05
2         ASN              19              3  2.845737e-04
19        TYR              60             35  2.117094e-03
11        LEU              10             29  2.690370e-03
14        PHE               8              2  5.701560e-02
0         ALA               1              7  6.900524e-02
12        LYS               5             13  9.269268e-02
6         GLN               4             11  1.151757e-01
3         ASP               6              2  1.661058e-01
1         ARG               4             10  1.758993e-

SER, TRP, GLY, TYR → häufiger im Paratope → wichtig für Antikörperbindung.

PRO, MET, GLU → häufiger im Epitope → typisch für das Antigen.